# Wrangling for Tidy Data

> 80% of the effort in data analysis is spent cleaning data ([Hadley Wickham](http://vita.had.co.nz/papers/tidy-data.pdf))

The work required to make your dataset easy to analyze.

- Easy to use column names
- Fixing inconsistent variables
- Merging with other datasets
- Reshaping (melt, pivot, ect..)
- Dealing with missing values

## Tidy data

- [Hadley's paper](http://vita.had.co.nz/papers/tidy-data.pdf)
- [Lecture for R users](http://vita.had.co.nz/papers/tidy-data-pres.pdf)

General Idea:

- Each variable forms a column
- Each row is an observation
- Each table stores a single type of observation type

> A general rule of thumb is that it is easier describe a functional relationships between columns (variables) than between rows, and it is easier to make comparisions between groups of rows (observations) than between columns.  -- Hadley, Tidy Data

**Argument**: Tidy data makes it easier to *manipulate*, *visualize*, and *model* your data.

## Example

Taken from [Hadley's paper](http://vita.had.co.nz/papers/tidy-data.pdf)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
names = ['John Smith','Jane Doe','Mary Johnson']
treatmentA = [None,1,2]
treatmentB = [5,4,3]

messy = pd.DataFrame({'name': names, 'A': treatmentA, 'B': treatmentB})
print(messy)

What are the variables in this experiment?

What are the observations?

In [ ]:
tidy = pd.melt(messy, id_vars=['name'])
tidy

In [ ]:
tidy.columns = ['name','treatment','result']
tidy

## "Unpivoting" data

A common type of messy dataset is tabular data designed for presentation, where variables form both the rows and columns, and column headers are values, not variable names. While we would call this arrangement messy, in some cases it can be extremely useful.

For example, the [World Bank](http://data.worldbank.org/indicator/SP.DYN.LE00.IN) usually provides CSV / XLS access to  data in “pivoted” or “normalized” form so they look like this:

### Pivoted data 
<img src="Pivoted.png">

But for a lot analyses and applications, particularly data visualisation tools like D3, ggplot2, Tableau, it’s more convenient to have your data “unpivoted” or “denormalized” so it looks like this:

### Unpivoted data
<img src="Unpivoted.png">

This is consistent with creating Tidy Data. Each variable forms a column, and each row is an observation. We want to “unpivot” or “denormalize” the data - in effect, to transpose columns to rows and have one complete record per row.

The pivoted format is often referred to as "wide," while the unpivoted format is called "long" or "tall" or "stacked."

### Preparing Excel Files

Tableau also provides a good tutorial about ["Preparing Excel Files for Analysis"](http://kb.tableausoftware.com/articles/knowledgebase/preparing-excel-files-analysis). That tutorial includes good general information about cleaning Excel files.

### Unpivoting with Excel (Windows only)

For Windows PCs, there is a convenient and free [Tableau Excel add-in](http://kb.tableausoftware.com/articles/knowledgebase/addin-reshaping-data-excel) or reshaping data. 

This approach is fine if you have Windows and Excel and are working on one file at a time, but many of us don’t use these tools and want tools that fit our own workflows better.

### Unpivoting with Pandas

A good way to achieve the desired result in Pandas is to use the “melt” function (seen briefly above). It is very similar to the "melt" function in the R reshape library that Hadley Wickham created for Tidy Data.

Let's use the full [dataset](http://data.worldbank.org/indicator/SP.DYN.LE00.IN) about life expectancy from birth from the World Bank. (Note that I cleaned it up a bit after the initial download. I removed two "title" rows and two unhelpful columns.)

In [ ]:
df = pd.read_csv('LifeExpectancy.csv')

In [ ]:
df.info()

In [ ]:
df.head()

Now `melt` the normalized file, hold the country name and code variables, and rename the melted columns.

In [ ]:
le = pd.melt(df, id_vars=['Country Name','Country Code'], var_name="year", value_name="life_expectancy")

In [ ]:
le.info()

In [ ]:
le.head()

In [ ]:
le.tail()

Sort by `Country Name` for convenience.

In [ ]:
le =  le.sort_values(['Country Name'])

In [ ]:
le.head()

In [ ]:
le.tail()

## Save the Tidy Data version

(Saving without an index)

In [ ]:
le.to_csv('LifeExpectancy-Tidy.csv', sep=',', index=False)

In [ ]:
!ls

## Speed Camera data example

From Jeff Leek's [Data Analysis Course](https://github.com/jtleek/dataanalysis)

[Open Baltimore](https://data.baltimorecity.gov) provides free public access to information about speed cameras (and many other things). Let's look at data from [here](https://data.baltimorecity.gov/Transportation/Baltimore-Fixed-Speed-Cameras/dz54-2aru).

While I have downloaded the dataset as '`cameras.csv`' and made it available for class, it is also possible to access it directly from the Web site. We call this "scraping." We'll do more Web scraping next week.

In [ ]:
url = 'https://data.baltimorecity.gov/api/views/dz54-2aru/rows.csv?accessType=DOWNLOAD'
cameras = pd.read_csv(url)

In [ ]:
cameras.head()

If, for some reason, that doesn't work, just uncomment the lines below to read the file.

In [ ]:
# cameras = pd.read_csv('cameras.csv')
# cameras.head()

Let's make the columns more standard.

In [ ]:
cameras.columns

In [ ]:
new_names = [ x.lower() for x in cameras.columns ]
new_names

In [ ]:
new_names = [ x.lower().split()[0] for x in cameras.columns ]
new_names

Putting it all together...

In [ ]:
cameras.columns = [ x.lower().split()[0] for x in cameras.columns ]
cameras.head()

### What if we want to split location into latitude and longitude?

Method one: `map` on location.

In [ ]:
def get_lat(x):
    tmp = x.replace('(','')
    tmp = tmp.replace(')','')
    lat = float(tmp.split(',')[0])
    return lat

In [ ]:
lat = cameras['location'].map(get_lat)
lat.head()

In [ ]:
tmp = cameras.copy()
tmp['lat'] = lat
tmp.head()

In [ ]:
def rt(x):
    return (1,2,3)

In [ ]:
tmp['x'] = cameras['location'].map(rt)
type(tmp.loc[0]['x'])

Method two: `apply` on the location.

In [ ]:
def split_location(x):
    tmp = x.replace('(','')
    tmp = tmp.replace(')','')
    val = pd.Series(tmp.split(',')).astype(np.float)
    return val

In [ ]:
tmp = cameras['location'].apply(split_location)
tmp.columns = ['lat','long']
print(type(tmp))
tmp.head()

Merge them togther.

In [ ]:
final = pd.concat([cameras,tmp], axis=1)
final.head()

The "melted" version looks like this:

In [ ]:
tidy = pd.melt(final[['crossstreet','street','lat','long']], id_vars=['crossstreet','street']).head()
tidy